# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Libs utilizadas:
- Selenium
- Pandas

In [1]:
# Imports

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime


### Coletar as novas cotações do Dólar, Euro e do Ouro

In [2]:
# Criar o navegador
navegador = webdriver.Edge()
navegador.maximize_window()


# Pesquisar por "Cotação Dólar"
navegador.get('https://google.com')  # Entrar no Google
search_box = navegador.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_box.send_keys('Cotação Dólar', Keys.ENTER)

# Pegar a cotação do Dólar
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]')
cotacao_dolar = cotacao_dolar.get_attribute('data-value')


# Pesquisar por "Cotação Euro"
navegador.get('https://google.com')
search_box = navegador.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_box.send_keys('Cotação Euro', Keys.ENTER)

# Pegar a cotação do Euro
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]')
cotacao_euro = cotacao_euro.get_attribute('data-value')


# Entrar no site Melhor Cambio
navegador.get('https://www.melhorcambio.com/ouro-hoje')

# Pegar a cotação do Ouro
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')

navegador.close()

print(datetime.now())
print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)

2023-01-20 12:20:06.261347
5.2055
5.6384
322,49


### Atualizando os preços e o cálculo do Preço Final

#### Passo 1: Importar a base de dados

In [3]:
tabela = pd.read_excel('Produtos.xlsx')

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


#### Passo 2: Atualizar as cotações

In [4]:
# Dolar
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = cotacao_dolar

# Euro
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = cotacao_euro

# Ouro
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = cotacao_ouro.replace(',', '.')

# Corrigir a tabela
tabela['Cotação'] = pd.to_numeric(tabela['Cotação'])

#### Passo 3: Atualizar o Preço de Compra e o Preço de venda

In [5]:
# Atualizar Preço de Compra (Preço de Compra = Preço Original * Cotação)
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# Atualizar Preço de venda (Preço de Venda = Preço de Compra * Margem)
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.2055,5205.447945,1.40,7287.627123
1,Carro Renault,4500.00,Euro,5.6384,25372.800000,2.00,50745.600000
2,Notebook Dell,899.99,Dólar,5.2055,4684.897945,1.70,7964.326506
3,IPhone,799.00,Dólar,5.2055,4159.194500,1.70,7070.630650
4,Carro Fiat,3000.00,Euro,5.6384,16915.200000,1.90,32138.880000
5,Celular Xiaomi,480.48,Dólar,5.2055,2501.138640,2.00,5002.277280
6,Joia 20g,20.00,Ouro,322.4900,6449.800000,1.15,7417.270000


### Exportar a nova base de preços atualizada

In [6]:
date = datetime.now()
data = f'{date.day}-{date.month}-{date.year}-{date.hour}-{date.minute}-{date.second}'
name = f'Produtos_atualizados_{data}.xlsx'

tabela.to_excel(name, index=False, startrow=0, sheet_name='Produtos')
print('Planilha criada: ', name)

Planilha criada:  Produtos_atualizados_20-1-2023-12-20-6.xlsx
